In [1]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> ""
## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [2]:
# Setting a temp directory for tests
TMP=/tmp/test-py-commands
## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
alias | wc -l

11


In [3]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3443)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3443

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-py-commands/test-3443


In [4]:
export PYTHON_CMD="/usr/bin/time python -u"
export PYTHON="$NICE $PYTHON_CMD"
export PYTHONPATH="$HOME/python:$PYTHONPATH"

In [5]:
## add-python-path(pkg-dir): add PKG-DIR to PATH and PARENT to PYTHONPATH

function add-python-path () {
    local package_path="$1"
    local parent_path=$(realpath "$package_path/..")
    prepend-path "$package_path"
    export PYTHONPATH="$parent_path:$PYTHONPATH"
}

alias mezcla-devel='add-python-path $HOME/python/Mezcla/mezcla'
alias mezcla-main='add-python-path $HOME/python/Mezcla-main/mezcla'
alias mezcla-tom='add-python-path $HOME/python/Mezcla-tom/mezcla'

if [[ ! "$PATH" =~ mezcla ]]; then
    true
fi

In [6]:
## FUNCTIONS REQUIRED FOR SMOOTH RUNNING OF THE TESTS
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }

function show-path-dir () { (echo "${1}:"; printenv "$1" | perl -pe "s/:/\n/g;") | $PAGER; }
alias show-path='show-path-dir PATH'

function append-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="${PATH}:$1"; fi }
function prepend-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="$1:${PATH}"; fi }

function append-python-path () { export PYTHONPATH=${PYTHONPATH}:"$1"; }
function prepend-python-path () { export PYTHONPATH="$1":${PYTHONPATH}; }
 

In [7]:

add-python-path $temp_dir
alias ps-python-full='ps-all python'

alias ps-python='ps-python-full | $EGREP -iv "(screenlet|ipython|egrep|update-manager|software-properties|networkd-dispatcher)"'
alias show-python-path='show-path-dir PYTHONPATH'

In [8]:
## ERROR 1 - process ID list syntax error

# $ ps-python-full
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).

In [9]:
## ERROR 2 - bash : command not found and process ID list syntax error
# $ ps-python
# | bash: -iv: command not found
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).


In [10]:
## show-python-path DISPLAYS ALL THE AVAILABLE PATHS OF PYTHON
show-python-path

PYTHONPATH:
/tmp/test-py-commands
/home/aveey/python



In [11]:
function python-lint-work() { python-lint-full "$@" 2>&1 | $EGREP -v '\((bad-continuation|bad-option-value|fixme|invalid-name|locally-disabled|too-few-public-methods|too-many-\S+|trailing-whitespace|star-args|unnecessary-pass)\)' | $EGREP -v '^(([A-Z]:[0-9]+)|(Your code has been rated)|(No config file found)|(\-\-\-\-\-))' | $PAGER; }

function python-lint() { python-lint-work "$@" 2>&1 | $EGREP -v '(Exactly one space required)|\((bad-continuation|bad-whitespace|bad-indentation|bare-except|c-extension-no-member|consider-using-enumerate|consider-using-with|global-statement|global-variable-not-assigned|keyword-arg-before-vararg|len-as-condition|line-too-long|logging-not-lazy|misplaced-comparison-constant|missing-final-newline|redefined-variable-type|redundant-keyword-arg|superfluous-parens|too-many-arguments|too-many-instance-attributes|trailing-newlines|useless-\S+|wrong-import-order|wrong-import-position)\)' | $PAGER; }


In [12]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint-work $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [13]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [14]:
# (REQUIRED BY run-python-lint-batched)

function downcase-stdin { perl -pe "use open ':std', ':encoding(UTF-8)'; s/.*/\L$&/;"; }
function downcase-text() { echo "$@" | downcase-stdin; }

function todays-date() { date '+%d%b%y' | downcase-stdin; }

function get-python-lint-dir () {
    local python_version_major=$(pylint --version 2>&1 | extract_matches.perl "Python (\d)")
    local affix="py${python_version_major}"
    local out_dir="_pylint/$(todays-date)-$affix"
    echo "$out_dir"
}

function run-python-lint-batched () {
    local file_spec="$@"
    if [ "$file_spec" = "" ]; then file_spec="*.py"; fi

    local out_dir=$(get-python-lint-dir)
    mkdir -p "$out_dir"

    (for f in $($LS $file_spec); do

         local b=$(basename "$f")
         local pre=""

         if [[ $f =~ / ]]; then pre="$(basename $(dirname "$f"))-"; fi
         DEBUG_LEVEL=5 python-lint "$f" >| "$out_dir/$pre$b".log 2>&1
         head "$out_dir/$pre$b".log
     done) >| "$out_dir/summary.log"
    less -p '^\** Module' "$out_dir/summary.log";
}

In [15]:
# ERROR: Pattern not found  (press RETURN)
# run-python-lint-batched ./print.py 

In [16]:
# python-import-path(module): find path for package directory of MODULE

function python-import-path-all() { local module="$1"; python -u -v -c "import $module" 2>&1; }
function python-import-path-full() { local module="$1"; python-import-path-all "$@" | extract_matches.perl "((matches (.*\W$module[^/]*[/\.][^/]*))|ModuleNotFoundError)"; }
function python-import-path() { python-import-path-full "$@" | head -1; }

# mezcla MODULE IS USED FOR TEST

python-import-path 'mezcla'
linebr
python-import-path-full 'mezcla'
linebr
# WORKS WELL, SHOWS SOME ISSUES ON BATSPP 2.1.X
python-import-path-all 'mezcla' | grep 'mezcla' | head -n 5

matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/__init__.py
--------------------------------------------------------------------------------
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/__init__.py
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/debug.py
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/sys_version_info_hack.py
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/glue_helpers.py
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/system.py
matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/tpo_common.py
--------------------------------------------------------------------------------
# /home/aveey/.local/lib/python3.10/site-packages/mezcla/__pycache__/__init__.cpython-310.pyc matches /home/aveey/.local/lib/python3.10/site-packages/mezcla/__init__.py
# code object from '/home/aveey/.local/lib/python3.10/site-packages/mezcla/__pycache__/__init__.cpython-310.pyc'
# /home/aveey/.lo

In [17]:
# RETURNS A LIST OF PYTHON MODULES
rm -rf ./*
pip3 freeze

anyio==3.6.1
appdirs==1.4.4
apturl==0.5.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
astroid==2.9.3
asttokens==2.0.8
attrs==22.1.0
ayatana-settings==21.1.28
Babel==2.10.3
backcall==0.2.0
bash_kernel==0.8.0
batspp==2.1.4
beautifulsoup4==4.11.1
beniget==0.4.1
bleach==5.0.1
blinker==1.4
Brlapi==0.8.3
Brotli==1.0.9
bs4==0.0.1
cajarename==21.11.24
certifi==2022.9.14
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.1.1
click==8.0.3
colorama==0.4.4
command-not-found==0.3
cryptography==3.4.8
cupshelpers==1.0
cycler==0.11.0
dbus-python==1.2.18
debugpy==1.6.3
decorator==5.1.1
defer==1.0.6
defusedxml==0.7.1
distro==1.7.0
distro-info===1.1build1
entrypoints==0.4
executing==1.0.0
fastjsonschema==2.16.1
folder-color-caja==0.0.86
folder-color-common==0.0.86
fonttools==4.29.1
fs==2.4.12
gast==0.5.2
gpg===1.16.0-unknown
greenlet==1.1.2
html5lib==1.1
httplib2==0.20.2
idna==3.4
importlib-metadata==4.6.4
ipykernel==6.15.3
ipython==8.5.0
ipython_genutils==0.2.0
ipywidgets==6.0.0
isort==5.6.4
jedi==0

In [18]:
# note: gotta hate python!
function python-module-version-full { local module="$1"; python -c "import $module; print([v for v in [getattr($module, a, '') for a in '__VERSION__ VERSION __version__ version'.split()] if v][0])"; }

function python-module-version { python-module-version-full "$@" 2> /dev/null; }
function python-package-members() { local package="$1"; python -c "import $package; print(dir($package));"; }
#
alias python-setup-install='log=setup.log;  rename-with-file-date $log;  uname -a > $log;  python setup.py install --record installed-files.list >> $log 2>&1;  ltc $log'

alias python-uninstall-setup='cat installed-files.list | xargs /bin/rm -vi; rename_files.perl -regex ^ un installed-files.list'

# MODULE USED: mezcla
python-module-version-full mezcla
linebr
python-module-version mezcla
linebr
python-package-members mezcla
linebr

1.3.1
--------------------------------------------------------------------------------
1.3.1
--------------------------------------------------------------------------------
['PYTHON3_PLUS', 'TL', '__VERSION__', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'debug', 'gh', 'glue_helpers', 'mezcla', 'sys', 'sys_version_info_hack', 'system', 'tpo_common']
--------------------------------------------------------------------------------


In [19]:
## rename-with-file-date REQUIRED
function rename-with-file-date() {

    local f new_f
    local move_command="move"
    if [ "$1" = "--copy" ]; then

        move_command="command cp --interactive --verbose --preserve"
        shift
    fi
    for f in "$@"; do
    
        if [ -e "$f" ]; then
           new_f=$(get-free-filename "$f".$(date --reference="$f" '+%d%b%y') ".")

           eval "$move_command" "$f" "$new_f";
        fi
    done;

}

function get-free-filename() {
    local base="$1"
    local sep="$2"
    local L=1
    local filename="$base"
    while [ -e "$filename" ]; do
        let L++
        filename="$base$sep$L"
    done;
    echo "$filename"
}

alias move='mv'
export PAGER=less
export PAGER_CHOPPED="less -S"
export PAGER_NOEXIT="less -+F"
function zless () { zcat "$@" | $PAGER; }

function zhead () {
    local file="$1"
    shift
    zcat "$file" | head "$@"
}
alias less-='$PAGER_NOEXIT'
alias less-clipped='$PAGER_NOEXIT -S'
alias less-tail='$PAGER_NOEXIT +G'
alias less-tail-clipped='$PAGER_NOEXIT +G -S'
alias ltc=less-tail-clipped


In [20]:
## REQUIRES setup.py (TBC)

# python-setup-install
# linebr
# python-uninstall-setup

In [30]:
alias set-xterm-window='set_xterm_title.bash'

function ipython() { 
    local ipython=$(which ipython)
    if [ "$ipython" = "" ]; then echo "Error: install ipython first"; return; fi
    set-xterm-window "ipython [$PWD]"
    $ipython '$@'
}

function python-trace {
    local script="$1"
    shift
    $PYTHON -m trace --trace $(which "$script") "$@"
    }
    
ipython --version | head
linebr
# STATEMENT TRACING WORKS WELL
python-trace l $BIN_DIR/tests/batspp_report.py | head

/home/aveey/bin/set_xterm_title.bash: line 239: /_set_xterm_title.27807.full.list: Permission denied
/home/aveey/bin/set_xterm_title.bash: line 240: /_set_xterm_title.27807.icon.list: Permission denied
[TerminalIPythonApp] WARNING | File '$@' doesn't exist
]1;ipython [/tmp/test-py-commands/test-3443]]2;ipython [/tmp/test-py-commands/test-3443]]0;IPython: test-py-commands/test-3443--------------------------------------------------------------------------------
 --- modulename: batspp_report, funcname: <module>
batspp_report.py(5): """ SELECTS AN IPYNB FILE,
batspp_report.py(11): import os
batspp_report.py(13): TEST_PATH = "./"
batspp_report.py(14): IPYNB_EXTENSION = ".ipynb"
batspp_report.py(15): BATSPP_EXTENSION = ".batspp"
batspp_report.py(16): i = 1
batspp_report.py(18): files_TEST_PATH = os.listdir(TEST_PATH)
batspp_report.py(21): all_testfiles = []
batspp_report.py(22): print("\n=== BATSPP REPORT GENERATOR (simple_batspp.py) ===\n")
Cannot run file '/home/aveey/tom-project/sh